In [1]:
from urllib import request
import os
import tarfile
import glob

In [2]:
filename,_=request.urlretrieve('https://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz', 'flowers.tgz')

In [3]:
tar=tarfile.open(filename)

In [4]:
tar.extractall(path='train_data')

In [5]:
tar.close

<bound method TarFile.close of <tarfile.TarFile object at 0x7e9689667340>>

In [6]:
image_paths = sorted(glob.glob('train_data/jpg/*.jpg'))

In [7]:
class_names = [
    'Buttercup', 'ColtsFoot', 'Daffodil', 'Daisy', 'Dandelion', 'Fritillary', 'Iris', 'Pansy', 'Sunflower', 'Windflower',
    'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus', 'Tigerlily', 'Tulip', 'Cowslip'
]
l=0
i=0
j=80
for k in range(1, 18):
    os.makedirs('train_data/' + class_names[l])
    current_path = 'train_data/' + class_names[l] + '/'
    for index, image_path in enumerate(image_paths[i:j], start=1):
        original = image_path
        image_path = image_path.split('/')
        image_file_name = str(index) + '.jpg'
        os.rename(original, current_path + image_file_name)
    l += 1
    i += 80
    j += 80


In [8]:
import shutil
source_folder = 'train_data/jpg'
destination_folder = 'JPG/jpg'
os.makedirs('JPG', exist_ok=True)
shutil.move(source_folder, destination_folder)
print(f'پوشه {source_folder} به {destination_folder} جابجا شد.')

پوشه train_data/jpg به JPG/jpg جابجا شد.


In [9]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import pandas as pd
import numpy as np
import cv2
import h5py

In [10]:
train_labels=os.listdir('train_data/')
train_labels.sort()

In [11]:
X = []
y = []
for label in train_labels:
    dir = os.path.join('train_data', label)
    current_label = label
    print('Current Folder: ' + current_label)
    for x in range(1, 81):
        print('Current Image: ' + str(x))
        file = os.path.join(dir, str(x) + '.jpg')
        image = cv2.imread(file)
        image = cv2.resize(image, (500, 500))
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        histogram_features = hist.flatten()
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hue_features = cv2.HuMoments(cv2.moments(image_gray)).flatten()
        current_features = np.hstack([histogram_features, hue_features])
        X.append(current_features)
        y.append(current_label)

Current Folder: Bluebell
Current Image: 1
Current Image: 2
Current Image: 3
Current Image: 4
Current Image: 5
Current Image: 6
Current Image: 7
Current Image: 8
Current Image: 9
Current Image: 10
Current Image: 11
Current Image: 12
Current Image: 13
Current Image: 14
Current Image: 15
Current Image: 16
Current Image: 17
Current Image: 18
Current Image: 19
Current Image: 20
Current Image: 21
Current Image: 22
Current Image: 23
Current Image: 24
Current Image: 25
Current Image: 26
Current Image: 27
Current Image: 28
Current Image: 29
Current Image: 30
Current Image: 31
Current Image: 32
Current Image: 33
Current Image: 34
Current Image: 35
Current Image: 36
Current Image: 37
Current Image: 38
Current Image: 39
Current Image: 40
Current Image: 41
Current Image: 42
Current Image: 43
Current Image: 44
Current Image: 45
Current Image: 46
Current Image: 47
Current Image: 48
Current Image: 49
Current Image: 50
Current Image: 51
Current Image: 52
Current Image: 53
Current Image: 54
Current Imag

In [13]:
np.shape(X)

(1360, 519)

In [14]:
np.shape(y)

(1360,)

In [16]:
le=LabelEncoder()
y=le.fit_transform(y)

In [17]:
mms=MinMaxScaler()
X=mms.fit_transform(X)

In [30]:
h5_x=h5py.File('data/x.h5','w')
h5_x.create_dataset('features',data=np.array(X))

<HDF5 dataset "features": shape (1360, 519), type "<f8">

In [23]:
h5_y=h5py.File('data/y.h5' , 'w')
h5_y.create_dataset('targets',data=np.array(y))

<HDF5 dataset "targets": shape (1360,), type "<i8">

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [31]:
x=np.array(h5py.File(os.path.join('data','x.h5') , 'r')['features'])
y=np.array(h5py.File(os.path.join('data','y.h5'), 'r')['targets'])

In [32]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=123)

In [33]:
rfc=RandomForestClassifier(n_estimators=400)
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.50      0.63      0.56        19
           1       0.52      0.75      0.62        16
           2       0.71      0.71      0.71        24
           3       0.80      0.87      0.83        23
           4       0.62      0.37      0.47        27
           5       0.67      0.50      0.57        20
           6       0.82      0.70      0.76        20
           7       0.65      0.73      0.69        15
           8       0.42      1.00      0.59        11
           9       0.90      0.79      0.84        24
          10       0.74      0.64      0.68        22
          11       0.44      0.35      0.39        20
          12       0.92      0.71      0.80        17
          13       0.89      0.81      0.85        21
          14       0.62      0.56      0.59        27
          15       0.70      0.89      0.78        18
          16       0.72      0.81      0.76        16

    accuracy              

In [34]:
from sklearn.svm import SVC

In [36]:
svc=SVC()
svc.fit(x_train,y_train)
y_pred_svc=svc.predict(x_test)
print(classification_report(y_pred_svc,y_test))

              precision    recall  f1-score   support

           0       0.46      0.48      0.47        23
           1       0.13      0.25      0.17        12
           2       0.54      0.41      0.46        32
           3       0.24      0.86      0.38         7
           4       0.38      0.19      0.25        32
           5       0.47      0.32      0.38        22
           6       0.41      0.58      0.48        12
           7       0.18      0.23      0.20        13
           8       0.65      0.59      0.62        29
           9       0.57      0.40      0.47        30
          10       0.58      0.48      0.52        23
          11       0.31      0.20      0.24        25
          12       0.69      0.39      0.50        23
          13       0.63      1.00      0.77        12
          14       0.25      0.43      0.32        14
          15       0.39      0.64      0.49        14
          16       0.56      0.59      0.57        17

    accuracy              

In [37]:
import xgboost as xgb

In [39]:
xgb_clf=xgb.XGBClassifier(n_estimators=400 , random_state=123 , )
xgb_clf.fit(x_train,y_train)
y_pred_xgb=xgb_clf.predict(x_test)
print(classification_report(y_pred_xgb, y_test))

              precision    recall  f1-score   support

           0       0.46      0.55      0.50        20
           1       0.43      0.56      0.49        18
           2       0.79      0.70      0.75        27
           3       0.80      0.95      0.87        21
           4       0.62      0.37      0.47        27
           5       0.47      0.54      0.50        13
           6       0.82      0.78      0.80        18
           7       0.71      0.71      0.71        17
           8       0.50      1.00      0.67        13
           9       0.81      0.77      0.79        22
          10       0.63      0.55      0.59        22
          11       0.44      0.28      0.34        25
          12       0.92      0.86      0.89        14
          13       0.89      0.77      0.83        22
          14       0.50      0.57      0.53        21
          15       0.74      0.85      0.79        20
          16       0.72      0.65      0.68        20

    accuracy              

In [40]:
from sklearn.model_selection import RandomizedSearchCV

In [52]:
xgb_clf_gsh=xgb.XGBClassifier()
params={'max_depth':list((range(3,8))) , 'alpha':[0 , 0.01 , 0.1 , 1] ,
        'n_estimators':[100,200,300,400,500] , 'learning_rate':np.linspace(0.1 , 0.5 , 10)}
gsh=RandomizedSearchCV(estimator=xgb_clf_gsh , param_distributions=params , cv=3 , verbose=2 , n_iter=5 , random_state=123)

In [53]:
gsh.fit(x,y)
print(gsh.best_params_)
print(gsh.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END alpha=0.1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=  33.5s
[CV] END alpha=0.1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=  30.1s
[CV] END alpha=0.1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=  31.2s
[CV] END alpha=0.01, learning_rate=0.2777777777777778, max_depth=6, n_estimators=100; total time=  17.3s
[CV] END alpha=0.01, learning_rate=0.2777777777777778, max_depth=6, n_estimators=100; total time=  17.6s
[CV] END alpha=0.01, learning_rate=0.2777777777777778, max_depth=6, n_estimators=100; total time=  15.0s
[CV] END alpha=0.01, learning_rate=0.32222222222222224, max_depth=4, n_estimators=300; total time=  23.2s
[CV] END alpha=0.01, learning_rate=0.32222222222222224, max_depth=4, n_estimators=300; total time=  22.5s
[CV] END alpha=0.01, learning_rate=0.32222222222222224, max_depth=4, n_estimators=300; total time=  19.1s
[CV] END alpha=0.01, learning_rate=0.188